In [14]:
import torch
from torch import nn
import torch.nn.functional as F

In [27]:
xs = []

In [44]:
class Hello(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, meta):
        ctx.meta = meta
        print("hello")
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        print("bw")
        print(ctx.meta)
        xs.append(1)
        return grad_input

In [45]:
x = torch.randn(2, 4, requires_grad=True)

In [46]:
f1 = nn.Linear(4, 2)

In [51]:
loss = f1(Hello.apply(x, 22))

hello


In [52]:
loss.sum().backward()

bw
22


RuntimeError: function HelloBackward returned an incorrect number of gradients (expected 2, got 1)

In [49]:
xs

[]

In [50]:
x.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [53]:
import torch.distributed as dist
import torch.distributed.rpc as rpc

In [ ]:
expert, router, loss function

In [ ]:
(current_rank - 1) % local_world_size

In [ ]:
local_max = torch.max(xs, dim=-1)
global_max = dist.all_reduce(local_max, op=dist.ReduceOp.MAX)
normalized_xs = xs - global_max

In [56]:
from einops import rearrange

In [57]:
class _ParallelCrossEntropy(torch.autograd.Function):
    @staticmethod
    def forward(ctx, parallel_logits, targets, parallel_context):
        rank = parallel_context.get_local_rank(ParallelMode.TENSOR)
        world_size = parallel_context.get_world_size(ParallelMode.TENSOR)
        
        partition_size = parallel_logits.shape[0] // world_size
        vocab_start_idx = rank*partition_size
        vocab_end_idx = vocab_start_idx + partition_size
        
        mask_targets = (targets < vocab_start_idx) | (targets >= vocab_end_idx)
        masked_targets = targets.clone() - vocab_start_idx
        masked_targets[mask_target] = 0
        
        masked_targets_1d = rearrange(
            masked_targets,
            "batch_size seq_len -> (batch_size seq_len)"
        )
        predicted_logits = parallel_logits[range(masked_targets.shape[0]), masked_targets_1d]
        predicted_logits = torch.where(mask_targets, 0, predicted_logits)
        
        predicted_logits = all_reduce(predicted_logits, op=dist.ReduceOp.MAX, parallel_context=parallel_context)
        exp_logits = torch.exp(parallel_logits).sum(dim=-1)
        loss = exp_logits.log() - predicted_logits
        return loss

In [55]:
class ParallelCrossEntropy(nn.Module):
    def __init__(self, parallel_context):
        super().__init__()
        self.parallel_context = parallel_context
    
    def forward(self, logits, targets):
        loss = _ParallelCrossEntropy.apply(logits, targets, self.parallel_context)
        loss /= len(targets)
        return loss

In [ ]:
def send_package(package, src, dst, parallel_context):
    dst_worker_name = parallel_context.get_worker_name(dst)
    
    rpc.rpc_sync(
        to=dst_worker_name
        fuc=recv_package,
        args=(package, src, dst)
    )

In [ ]:
def recv_package(package, src, dst):
    RECV_QUEUE.put(package)

In [58]:
fut = torch.futures.Future()

In [59]:
def f(fut):
    print(6942)

In [ ]:
fut.add_done_callback(f)

In [ ]:
fut.set_result(VALUE)

In [ ]:
step 1: parallelize qkv
step 2: acts
step 3: do local attention
step 4: 

In [ ]:
stream = torch.cuda.Stream(device)

In [ ]:
with torch.cuda.device(device):
    with torch.cuda.stream(stream):
        mean = xs.mean(dim=-1)

In [ ]:
step 1: register autograd
step 2: loss.backward()
step 3: torch trigger backward
step 4: schedule
step 5: execute

In [ ]:
step 1: profile
step 2: determine the number of partitions
step 3: split
step 4: move

In [ ]:
host dis, elastic driver, 

In [60]:
world_size = 16

In [61]:
tensor_model_parallel_size = 2

In [62]:
pipeline_model_parallel_size = 4

In [63]:
data_parallel_groups = []

In [ ]:
num_data_parallel_groups = world_size // da

In [ ]:
for i in range(pipeline_model_parallel_size):
    start_rank = i*pipeline_model_parallel_size
     

In [ ]:
- step 1: prob[0] = sigmoid(logit0 - logit1)
- step 2: logit0 = resid2[0] @ W_U[0], logit1 = resid2[0] @ W_U[1]
- step 3: logit0 - logit1 = resid2[0] @ (W_U[0] - W_U[1])
- step 4: resid2[0] = resid1[1] @ ln2 @ W_OV^{2, 0}
- step 5: logit0 - logit1 = resid1[1] @ ln2 @ W_OV^{2, 0} @ (W_U[0] - W_U[1])
- step 6: 

In [65]:
layer_idx, neuron_idx = 5, 1393

In [ ]:
W_in = model.W_in[layer_idx, :, neuron_idx]

In [66]:
from einops import einsum

In [ ]:
def compute_consine_similarity(w_in, feature):
    w_in /= w_in.norm(dim=-1)
    feature /= feature.norm(dim=-1)
    
    return einsum(w_in, feature, "")

In [ ]:
step 1: dest pos
step 2: identify which head 
step 3: where does it get information from
step 4: disrupt


In [2]:
from torch.utils.data import Dataset

In [3]:
class CachedDataset(Dataset):
    def __init__(self, filename):
        self.filename = filename
        self.cache = None
    
    def prefetch(self, idxs):
        data = torch.load(self.filename)
        total_elements = sum([data[idx].numel() for idx in idxs])
        
        self.cache = torch.zeros(total_elements, dtype=data.dtype)
        
        offset = 0
        for idx in idxs:
            n_elements = data[idx].numel()
            self.cache[offset:offset+total_elements] = data[idx].view(-1)
            offset += n_elements

In [4]:
world_size = 16

In [5]:
tensor_parallel_size = 2
pipeline_parallel_size = 4

In [6]:
num_pipeline_model_parallel_groups = world_size // pipeline_parallel_size

In [7]:
data_parallel_groups = []

In [9]:
for i in range(pipeline_parallel_size):
    start_rank = i*num_pipeline_model_parallel_groups
    end_rank = (i+1)*num_pipeline_model_parallel_groups
    
    for j in range(tensor_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_parallel_size
        ))
    
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [ ]:
applied force, gravity, normal force, air friction, friction

In [ ]:
two uncertain principles, quantization of action, angular momentum

In [ ]:
strong weak, gravity, electromagntic radiation